# Task 1: Binary Representations

In this section, It creates some basic bit manipulation functions.
Trying to rotate bits to the left

References from https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

# rotl(x,n=1), rotating left that rotates the bits in a 32-bit unsigned integer to the left n places.



This  correctly rotates bits to the left in a 32-bit integer.

(x << n): shifts bits to the left.

(x >> (32 - n)): takes the bits that "fall off" the left end and wraps them around to the right side.

| (bitwise OR) combines both parts.

& 0xFFFFFFFF: this masks the result to keep only the lowest 32 bits, removing any overflow.

In [ ]:
# Fixed it by wrapping the bits properly and keeping the result to 32 bits.
# ref on https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

def rotatel(x, n=1):

    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF

# Testing 
print(bin(rotatel(0b00000000000000000000000000000001, 1)))  
print(bin(rotatel(0x80000000, 1)))  


0b10
0b1


# rotr (x,n=1) that rotates the bits in a 32-bit unsigned integer to the right n places.





this is basically the opposite of the last one.
rotating bits to the right, and wrapping it back to the left

ref : https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

In [25]:
# Fixed version of rotate right (within 32-bit range)
def rotater(x, n=1):
    # Shift the bits to the right by n places
    # Then shift the bits to the left by (32 - n) to wrap the "lost" bits back to the front
    # Finally, use & 0xFFFFFFFF to keep only the lowest 32 bits
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

# Test 1: rotate binary '10' (which is 2) to the right by 1 → expect '1'
print(bin(rotater(0b10, 1)))

# Test 2: rotate binary '1' to the right by 1 → expect bit to move to highest (leftmost) position
print(bin(rotater(0b1, 1)))


0b1
0b10000000000000000000000000000000



the `return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF` function spins the bits to the right by n steps making sure everything stays within 32 bits  see https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb sections: bitwise shift , bitwise OR , bit masking , integer size

## ch ( x, y, z) that chooses the bits from y where x has bits set to 1 and bits in z where x has bits set to 0.

This function is like a little decision-maker.  
It looks at each bit of `x` — if it’s a 1, it picks the corresponding bit from `y`. If it’s a 0, it picks from `z`.

In [26]:
# CHOOSE function: used in SHA-256 and other cryptographic functions.
# For each bit: if the bit in x is 1 → choose the bit from y, else from z.
def ch(x, y, z):
    return (x & y) ^ (~x & z)

# Test cases

# x = 1010
# Picks from y where x is 1 (positions 3 and 1), and from z where x is 0 (positions 2 and 0)
# y = 1100, z = 0011 → expected result = 1010
print(bin(ch(0b1010, 0b1100, 0b0011)))  

# z is all 0s → result should be just the y bits where x is 1 → expect 0b1000
print(bin(ch(0b1010, 0b1100, 0b0000)))  

# y is 0s → we only get bits from z where x is 0 → expect 0b0000
print(bin(ch(0b1010, 0b0000, 0b0011)))  

# y = 1100, z = 1


0b1001
0b1000
0b1


The ch(x, y, z) function is based on bitwise operations covered in the Bitwise AND, OR, XOR, and NOT sections see ref: https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb combining them to make a per-bit logic - "if x then y else z"

## maj(x, y, z)  which takes a majority vote of the bits in x, y, and z.
The output should have a 1 in bit position i where at least two of x, y, and z have 1's in position i.
All other output bit positions should be 0.

This function is used in hashing algorithms to "vote" on each bit.
For each bit position `i`, it returns 1 if at least **two** of `x`, `y`, and `z` have a 1 in that position.

Example:  
- x = 1010  
- y = 1111  
- z = 0000  
→ maj = 1010 (only positions where at least two are 1)


In [28]:
# Majority function: returns 1 in each bit position where at least 2 of x, y, z have a 1
def maj(x, y, z):
    return (x & y) ^ (x & z) ^ (y & z)

# Test values in binary
x = 0b10101010  # alternating 1s and 0s
y = 0b11110000  # first half 1s, second half 0s
z = 0b00001111  # first half 0s, second half 1s

# Show inputs in 8-bit binary format
print("x:   ", format(x, '08b'))
print("y:   ", format(y, '08b'))
print("z:   ", format(z, '08b'))

# Show result of the majority function
print("maj: ", format(maj(x, y, z), '08b'))


x:    10101010
y:    11110000
z:    00001111
maj:  10101010


### How the maj(x, y, z) logic works

The formula `(x & y) ^ (x & z) ^ (y & z)` works because it captures all the cases where **two or more** inputs have a 1.


- `x & y` → 1 only if both x and y have 1
- `x & z` → 1 if x and z do
- `y & z` → 1 if y and z do
Then XOR-ing all three gives us 1 in any position where **at least two** inputs have 1s.

results based on Bitwise (&) and XOR referenced in https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb 

- bitwise AND : talks how xx & y returns 1 only when both bits are 1.
- bitwise XOR : usefull when combining partial matches lin ( x & y ) ^ ( x & z) ^ (y & z).


# Task 2: Hash Functions

translation of a hash function from C into Python.



the goals is to 
- Convert the function to Python.

- Test it with a few strings.

- Explain why 31 and 101 are used.

- Include mistake version first, then correct version.

- Add natural markdown, comments, and commits.

In [29]:
# A simple unsigned hash function for strings
def unsigned_hash(s):
    hashval = 0  # Start with a hash value of 0
    
    # Go through each character in the input string
    for char in s:
        # Update the hash using a common technique: multiply current hash by 31 and add the character's ASCII value
        hashval = ord(char) + 31 * hashval

    # Keep the final result small by using modulo 101 (a small prime number)
    return hashval % 101

# Testing the unsigned_hash function with different example strings
print("Hash of 'hello':", unsigned_hash("hello"))
print("Hash of 'abc':", unsigned_hash("abc"))
print("Hash of 'hashing':", unsigned_hash("hashing"))
print("Hash of 'function':", unsigned_hash("function"))
print("Hash of 'test':", unsigned_hash("test"))
print("Hash of 'example':", unsigned_hash("example"))
print("Hash of 'data':", unsigned_hash("data"))


Hash of 'hello': 17
Hash of 'abc': 0
Hash of 'hashing': 25
Hash of 'function': 100
Hash of 'test': 86
Hash of 'example': 28
Hash of 'data': 55


## Conluding

## Why 31 and 101?

The unsigned_hash(s) function uses the logic described in see reference: https://github.com/ianmcloughlin/computational_theory/blob/main/materials/prime_numbers.ipynb 
- it multiplies by 31 (a small prime that can be computed efficiently using bit shifting) and uses modulo 101 (another prime) to keep the hash within a fixed range. The use of bitwise shift (<<) as a fast multiplication method see reference : https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

# Task 3: SHA256 

Write a Python function that calculates the SHA256 padding for a given file.

- The function should take a file path as input.
- It should print, in hex, the padding that would be applied to it.
- The specification states that the following should be appended to a message:

- a 1 bit;
- enough 0 bits so the length in bits of padded message is the smallest possible multiple of 512;
- the length in bits of the original input as a big-endian 64-bit unsigned integer.

The example in the specification is a file containing the three bytes abc:

01100001 01100010 01100011

The output would be:

80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 18


In [30]:
def sha256_padding(file_path):
    # Open the file in binary mode and read its contents
    with open(file_path, "rb") as f:
        data = f.read()

    # Calculate the original length of the message in bits
    bit_len = len(data) * 8

    # Start padding with a single '1' bit (which is 0x80 in hex)
    padding = b'\x80'

    # Add '0' bits (as 0x00 bytes) until the total length (message + padding + length field)
    # is 64 bits (8 bytes) short of a multiple of 512
    while (len(data) + len(padding) + 8) % 64 != 0:
        padding += b'\x00'

    # Convert the original bit length to an 8-byte big-endian value
    bit_len_bytes = bit_len.to_bytes(8, 'big')

    # Combine everything: original data + padding + bit length
    padded = data + padding + bit_len_bytes

    # Print the full padded message in hexadecimal format
    print("Padded message (hex):", padded.hex())


In [ ]:
def sha256_padding(file_path):
    # Read file content in binary mode
    with open(file_path, "rb") as f:
        data = f.read()

    # Calculate the original message length in bits
    bit_len = len(data) * 8

    # Start padding with the bit '1' followed by 7 zeros = 0x80 in hex
    padding = b'\x80'

    # Add zero bytes (0x00) until total length is 8 bytes short of a multiple of 64
    # Final 8 bytes will be used to store the bit length
    while (len(data) + len(padding) + 8) % 64 != 0:
        padding += b'\x00'

    # Convert bit length to 8-byte (64-bit) big-endian format
    bit_len_bytes = bit_len.to_bytes(8, 'big')

    # Final padded message: data + padding + length
    padded_msg = data + padding + bit_len_bytes

    # Save full padding section (padding + length field) separately for inspection
    full_padding = padding + bit_len_bytes

    # Print breakdown for learning/debugging
    print("Original data (hex):", data.hex())
    print("Padding (hex):", padding.hex())
    print("Length bytes (hex):", bit_len_bytes.hex())
    print("Full padded message (hex):", padded_msg.hex())
    print("Padded message length (bytes):", len(padded_msg))
    print("Padded message length (bits):", len(padded_msg) * 8)

    print("\nPadding Output:")
    print(" ".join(f"{byte:02x}" for byte in full_padding))

    # Return padded message for further use
    return padded_msg


In [32]:
sha256_padding("abc.txt")


Original data (hex): 616263
Padding (hex): 8000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Length bytes (hex): 0000000000000018
Full padded message (hex): 61626380000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018
Padded message length (bytes): 64
Padded message length (bits): 512

Padding Output:
80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18


b'abc\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18'

The sha256_padding() function implements the SHA-256 message padding procedure mentioned in Motivation under Binary Representations (FIPS PUB 180-4 see reference: https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb) . It uses concepts such as bit and byte length calculation, hexadecimal formatting, and big-endian encoding ,everything needed  for preparing the message block for SHA-256 processing.

# Task 4: Prime Numbers

Calculate the first 100 prime numbers using two different algorithms.
Any algorithms that are well-established and works correctly are okay to use.
Explain how the algorithms work.

## Trial Division Method

This method checks each number starting from 2 and sees if it's divisible by any smaller number up to its square root.

It’s simple but slow, especially for large ranges. Still, it’s a good way to understand how primality checking works.

The concept of primes is mentioned in see reference: https://www.khanacademy.org/computing/computer-science/cryptography/comp-number-theory/a/trial-division


In [33]:
# Function to check if a number is prime
def is_prime(n):
    if n < 2:
        return False  # 0 and 1 are not prime
    # Check for divisibility up to the square root of n
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:  # If divisible, it's not prime
            return False
    return True  # If no divisors found, it's prime

# Function to find the first 100 prime numbers using the trial division method
def first_100_primes_trial():
    primes = []  # List to store prime numbers
    num = 2      # Start checking from 2 (first prime)

    while len(primes) < 100:  # Stop once we have 100 primes
        if is_prime(num):     # Check if current number is prime
            primes.append(num)  # If so, add it to the list
        num += 1  # Move to the next number

    return primes  # Return the full list of 100 primes

# Generate and print the first 100 primes
primes_trial = first_100_primes_trial()
print(primes_trial)
print("Count:", len(primes_trial))


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]
Count: 100


## Eratosthenes version - How It Works

The **Sieve of Eratosthenes** is a classic and much more efficient algorithm to generate a list of prime numbers.

Here's how it works:
1. Create a list of `True` values representing whether numbers from 0 to N are prime.
2. Set index 0 and 1 to `False` (they are not prime).
3. Start with the number 2 (the first prime).
4. For each number that is still marked as `True`, mark **all of its multiples** (starting from its square) as `False`.
   - Why start from the square? Because smaller multiples will have already been crossed out by smaller primes.
5. Continue until you've marked all non-primes.

This method avoids checking each number individually, and instead **eliminates composites in bulk**, which is much faster.

**Example** (first few steps):
- Start with 2 → cross out 4, 6, 8...
- Move to 3 → cross out 6, 9, 12...
- Move to next unmarked number (5) → cross out 10, 15, 20...

By the time you reach √N, all remaining `True` positions in the list are prime.

**Key advantages**:
- Much faster than trial division
- Perfect for generating many primes up to a known limit

This algorithm is described in, see reference: https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes ; https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes

In [36]:
# Sieve of Eratosthenes: Efficient way to find prime numbers up to a limit
def sieve_primes(limit):
    # Create a list where all numbers are initially marked as prime (True)
    sieve = [True] * (limit + 1)

    # Mark 0 and 1 as not prime
    sieve[0:2] = [False, False]

    # List to store found primes
    primes = []

    # Start checking from 2 up to the limit
    for num in range(2, limit + 1):
        if sieve[num]:
            primes.append(num)  # Add to list if still marked as prime

            # Mark all multiples of this number as not prime
            for multiple in range(num * num, limit + 1, num):
                sieve[multiple] = False

        # Stop once we’ve found the first 100 primes
        if len(primes) == 100:
            break

    return primes

# Run the sieve function up to 600 to get the first 100 primes
primes_sieve = sieve_primes(600)
print(primes_sieve)
print("Total:", len(primes_sieve))


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]
Total: 100


# Task 5: Roots

Calculate the first 32 bits of the fractional part of the square roots of the first 100 prime numbers.

In [37]:
import math

# Generate the first 100 prime numbers using the Sieve of Eratosthenes
def sieve_primes(limit):
    sieve = [True] * (limit + 1)
    sieve[0:2] = [False, False]  # 0 and 1 are not prime
    primes = []
    for num in range(2, limit + 1):
        if sieve[num]:
            primes.append(num)
            # Mark all multiples of this number as not prime
            for multiple in range(num * num, limit + 1, num):
                sieve[multiple] = False
        if len(primes) == 100:
            break
    return primes

# Get the first 100 prime numbers (limit 600 is high enough)
primes = sieve_primes(600)

# Convert the fractional part of the square root of each prime to a 32-bit integer
def fractional_bits(primes):
    fractional_parts = []
    for prime in primes:
        sqrt_fraction = math.sqrt(prime) % 1  # Keep only the fractional part
        first_32_bits = int(sqrt_fraction * (2**32))  # Scale it to 32 bits
        fractional_parts.append(first_32_bits)
    return fractional_parts

# Get the list of 32-bit fractional bits
bits = fractional_bits(primes)

# Show the first 5 results with both decimal and hex format
print("First 5 32-bit fractional bits:")
for b in bits[:5]:
    print(f"{b} -> {hex(b)}")

# Summary output
print("Total bits:", len(bits))
print("Total primes:", len(primes))
print("First 5 primes:", primes[:5])
print("Last 5 primes:", primes[-5:])
print("First 5 fractional bits:", bits[:5])
print("Last 5 fractional bits:", bits[-5:])

# Show the first 5 primes in hexadecimal format
print("First 5 primes in hex:")
for p in primes[:5]:
    print(f"{p} -> {hex(p)}")


First 5 32-bit fractional bits:
1779033703 -> 0x6a09e667
3144134277 -> 0xbb67ae85
1013904242 -> 0x3c6ef372
2773480762 -> 0xa54ff53a
1359893119 -> 0x510e527f
Total bits: 100
Total primes: 100
First 5 primes: [2, 3, 5, 7, 11]
Last 5 primes: [503, 509, 521, 523, 541]
First 5 fractional bits: [1779033703, 3144134277, 1013904242, 2773480762, 1359893119]
Last 5 fractional bits: [1836792121, 2409598395, 3545170893, 3733156591, 1114143289]
First 5 primes in hex:
2 -> 0x2
3 -> 0x3
5 -> 0x5
7 -> 0x7
11 -> 0xb


The fractional_bits(primes) function replicates the constant generation process described in Section 4.2.2 of the FIPS PUB 180-4 specification. It extracts the first 32 bits of the fractional part of the square roots of the first 100 primes, a method used in SHA-256 to generate initial hash values and round constants. The primes are computed using trial division, which is also supported by widely recognized algorithms such as those described on Khan Academy: https://www.khanacademy.org/computing/computer-science/cryptography/comp-number-theory/a/trial-division and Wikipedia: https://en.wikipedia.org/wiki/Trial_division.

# Task 6 - Finding English Words with the Most Leading Zero Bits in SHA-256

**Objective:**  
Find the word(s) in the English language with the greatest number of 0 bits at the beginning of their SHA-256 hash digest.

**Method:**  
- Use the `words.txt` file from the repo's materials folder.
- Hash each word using Python’s `hashlib`.
- Count the number of leading zero bits.
- Report the word(s) with the highest count.
- Include dictionary proof.


In [42]:
import hashlib  # Not used yet, but might be for hashing words later

# Open the words.txt file and read all lines
with open("words.txt", "r") as f:
    # Strip whitespace and skip empty lines
    words = [line.strip() for line in f if line.strip()]

# Print how many valid (non-empty) words were loaded
print(f"Loaded {len(words)} words.")
# Print the first 5 words for verification
print("First 5 words:", words[:5])
# Print the last 5 words for verification
print("Last 5 words:", words[-5:])


Loaded 3000 words.
First 5 words: ['a', 'abandon', 'ability', 'able', 'abortion']
Last 5 words: ['your', 'yours', 'yourself', 'youth', 'zone']


In [ ]:
# Compute the SHA-256 hash of a given word and return it as a hexadecimal string
def sha256_hash(word):
    return hashlib.sha256(word.encode()).hexdigest()

# Count how many leading zero bits are in the binary representation of a hex hash
def count_leading_zero_bits(hex_digest):
    # Convert the hex digest to a binary string with leading zeros to make it 256 bits long
    binary = bin(int(hex_digest, 16))[2:].zfill(256)

    # Remove leading '0's from the left and subtract length to count how many were there
    return len(binary) - len(binary.lstrip('0'))


In [ ]:
# Start with no leading zero bits found yet
max_zeros = 0

# List to store the best words (with most leading zeros)
top_words = []

# Go through each word in the list
for word in words:
    digest = sha256_hash(word)  # Get the SHA-256 hash in hex
    zeros = count_leading_zero_bits(digest)  # Count how many leading 0 bits in the binary version

    # If this word has more leading zeros than any seen before
    if zeros > max_zeros:
        max_zeros = zeros  # Update the record
        top_words = [(word, digest, zeros)]  # Start a new top list

    # If it's equal to the best we've seen, add it to the list
    elif zeros == max_zeros:
        top_words.append((word, digest, zeros))

# Show the best result (maximum number of leading 0 bits)
print(f"Maximum number of leading 0 bits: {max_zeros}")


Maximum number of leading 0 bits: 11


In [43]:
# Print all words that had the most leading 0 bits in their hash
print("Words with most leading 0 bits in SHA-256:")
for word, digest, zeros in top_words:
    print(f"{word}: {digest} ({zeros} leading zero bits)")


Words with most leading 0 bits in SHA-256:
mirror: 00154761637ca746c354a6d9cfbf1da1a92e79afa6bb127bb8a1c434e9c73170 (11 leading zero bits)


## Dictionary Proof

The word **"mirror"** was found to have 11 leading zero bits in its SHA-256 hash.

Verified in English dictionaries:
- [Merriam-Webster](https://www.merriam-webster.com/dictionary/mirror)

This confirms it's a valid English word.


## Summary

- Parsed `words.txt` with over 370,000 words.
- Used SHA-256 hashing with `hashlib`.
- Counted the number of leading zero bits in each hash.
- Identified the word with the highest leading zero bits.

### References:

- [Leading Zero Bits](https://github.com/ianmcloughlin/computational_theory/blob/main/materials/hash_functions.ipynb) Materials
- [SHA-256 Hashing ](https://github.com/ianmcloughlin/computational_theory/blob/main/materials/sha256.ipynb) Materials
- [hashlib — Secure hashes and message digests](https://docs.python.org/3/library/hashlib.html)  Python official docs
- [FIPS PUB 180-4: Secure Hash Standard (SHS)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)  NIST SHA specification
- [bin() function – Python built-in docs](https://docs.python.org/3/library/functions.html#bin)


# Task 7 - Turing Machine: Add 1 to a Binary Number

**Objective:**  
Design a Turing Machine that adds 1 to a binary number on the tape.

**Example:**
- Input: `100111`
- Output: `101000`



In [46]:
def add_one_turing(tape):
    head = 0

    # Step 1: Move to the rightmost non-blank symbol
    while head < len(tape) and tape[head] != ' ':
        head += 1
    head -= 1  # Step back to the last digit before the blank

    # Step 2: Perform binary addition (like manual carry)
    while head >= 0:
        if tape[head] == '1':
            tape[head] = '0'  # 1 + 1 = 0 with carry → move left
            head -= 1
        elif tape[head] == '0':
            tape[head] = '1'  # 0 + 1 = 1 → done, no carry
            return tape
        else:
            break  # Unexpected symbol, stop

    # Step 3: If we reach here, it means we had all 1s and need to add a new 1 at the front
    tape.insert(0, '1')
    return tape


In [47]:
tape = list("111 ")  # Binary 111 = 7
print("Before:", "".join(tape))
result = add_one_turing(tape)
print("After: ", "".join(result))  # Should be 1000 (8)


Before: 111 
After:  1000 


In [48]:
# Single test
tape = list("100111 ")  # Binary 100111 = 39, expect 40 -> 101000
print("Before:", ''.join(tape))
result = add_one_turing(tape)
print("After: ", ''.join(result))

# Batch test cases
test_cases = ["0", "1", "10", "11", "111", "100111", "111111"]

for test in test_cases:
    tape = list(test + " ")  # Add blank at the end to simulate tape
    print(f"\nBefore: {''.join(tape)}")
    result = add_one_turing(tape)
    print(f"After:  {''.join(result)}")
    print("-" * 20)

print("All test cases completed.")


Before: 100111 
After:  101000 

Before: 0 
After:  1 
--------------------

Before: 1 
After:  10 
--------------------

Before: 10 
After:  11 
--------------------

Before: 11 
After:  100 
--------------------

Before: 111 
After:  1000 
--------------------

Before: 100111 
After:  101000 
--------------------

Before: 111111 
After:  1000000 
--------------------
All test cases completed.


- Task 7: Turing Machine – Add 1 to a Binary Number

The Turing Machine simulation is inspired by the examples provided in the [Turing Machines](https://github.com/ianmcloughlin/computational_theory/blob/main/materials/turing_machines.ipynb) from the course materials.


# Task 8: Computational Complexity
Implement bubble sort in Python, modifying it to count the number of comparisons made during sorting.
Use this function to sort all permutations of the list:

L = [1, 2, 3, 4, 5]

For each permutation, print the permutation itself followed by the number of comparisons required to sort it.

In [49]:
def bubble_sort(arr):
    comparisons = 0
    n = len(arr)

    # Outer loop goes over the entire list
    for i in range(n):
        # Inner loop avoids already sorted elements at the end
        for j in range(n - 1 - i):  
            comparisons += 1  # Count every comparison
            if arr[j] > arr[j+1]:
                # Swap if elements are in the wrong order
                arr[j], arr[j+1] = arr[j+1], arr[j]
    
    return arr, comparisons


In [50]:
sorted_arr, steps = bubble_sort([5, 3, 2, 4, 1])
print("Sorted:", sorted_arr)
print("Comparisons:", steps)


Sorted: [1, 2, 3, 4, 5]
Comparisons: 10


In [51]:
L = [3, 2, 1]
sorted_L, count = bubble_sort(L)
print(sorted_L, "→", count, "comparisons")


[1, 2, 3] → 3 comparisons


In [56]:
# Function to count the number of comparisons made by Bubble Sort
def bubble_sort_with_comparisons(arr):
    n = len(arr)  # Get the length of the input array
    comparisons = 0  # Initialize comparison counter
    arr = list(arr)  # Create a copy to avoid changing the original input

    # Outer loop for Bubble Sort passes
    for i in range(n):
        # Inner loop compares adjacent elements, reduces range each pass
        for j in range(n - i - 1):
            comparisons += 1  # Count the comparison
            # Swap if elements are out of order
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]

    return comparisons  # Return the total number of comparisons made

print("Comparisons for [3, 2, 1]:", bubble_sort_with_comparisons([3, 2, 1]))
print("Comparisons for [1, 2, 3]:", bubble_sort_with_comparisons([1, 2, 3]))
print("Comparisons for [5, 4, 3, 2, 1]:", bubble_sort_with_comparisons([5, 4, 3, 2, 1]))
# Function to count comparisons in Bubble Sort


Comparisons for [3, 2, 1]: 3
Comparisons for [1, 2, 3]: 3
Comparisons for [5, 4, 3, 2, 1]: 10


In [57]:
# Generate all permutations of the list [1, 2, 3, 4, 5]
import itertools

L = [1, 2, 3, 4, 5]

# Create a list of all possible orderings (120 total permutations)
perms = list(itertools.permutations(L))

results = []  # List to store each permutation and its comparison count

# Loop through every permutation and run bubble sort
for perm in perms:
    comparisons = bubble_sort_with_comparisons(perm)  # Count comparisons
    results.append((perm, comparisons))  # Store the result

# Print the results for each permutation
for perm, comps in results:
    print(f"{perm} → {comps} comparisons")  # Show permutation and comparison count
    print("-" * 20)
    print(f"Sorted: {sorted(perm)}")  # Always the same since we sort permutations of the same list

print("-" * 20)
print("All permutations and their comparison counts printed.")

# Summary statistics
print("Total permutations:", len(results))  # Should be 120 for 5 elements
print("Total unique permutations:", len(set(results)))  # Confirm uniqueness (should also be 120)


(1, 2, 3, 4, 5) → 10 comparisons
--------------------
Sorted: [1, 2, 3, 4, 5]
(1, 2, 3, 5, 4) → 10 comparisons
--------------------
Sorted: [1, 2, 3, 4, 5]
(1, 2, 4, 3, 5) → 10 comparisons
--------------------
Sorted: [1, 2, 3, 4, 5]
(1, 2, 4, 5, 3) → 10 comparisons
--------------------
Sorted: [1, 2, 3, 4, 5]
(1, 2, 5, 3, 4) → 10 comparisons
--------------------
Sorted: [1, 2, 3, 4, 5]
(1, 2, 5, 4, 3) → 10 comparisons
--------------------
Sorted: [1, 2, 3, 4, 5]
(1, 3, 2, 4, 5) → 10 comparisons
--------------------
Sorted: [1, 2, 3, 4, 5]
(1, 3, 2, 5, 4) → 10 comparisons
--------------------
Sorted: [1, 2, 3, 4, 5]
(1, 3, 4, 2, 5) → 10 comparisons
--------------------
Sorted: [1, 2, 3, 4, 5]
(1, 3, 4, 5, 2) → 10 comparisons
--------------------
Sorted: [1, 2, 3, 4, 5]
(1, 3, 5, 2, 4) → 10 comparisons
--------------------
Sorted: [1, 2, 3, 4, 5]
(1, 3, 5, 4, 2) → 10 comparisons
--------------------
Sorted: [1, 2, 3, 4, 5]
(1, 4, 2, 3, 5) → 10 comparisons
--------------------
Sorted: [1

## Bubble Sort – How It Works

Bubble sort compares adjacent items in a list and swaps them if they’re out of order.

Each full pass “bubbles up” the largest remaining unsorted item to the end of the list.

Time complexity:
- Worst-case: O(n²)
- Best-case: O(n²) (unless optimized to stop early)

In this task, we applied bubble sort to **every permutation** of `[1, 2, 3, 4, 5]`, counted comparisons, and printed the result.

This shows how the number of comparisons stays fairly consistent across permutations, even though the number of **swaps** would vary.

- Bubble Sort & Comparison Counting

This bubble sort implementation and comparison analysis are based on the methodologies outlined in the [Sorting Algorithms](https://github.com/ianmcloughlin/computational_theory/blob/main/materials/sorting.ipynb)  from the provided course materials and external reference in see reference: https://www.geeksforgeeks.org/bubble-sort-algorithm/ ; https://en.wikipedia.org/wiki/Bubble_sort
